In [ ]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit,KFold
from os.path import join as opj
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pylab
plt.rcParams['figure.figsize'] = 10, 10

In [ ]:
#Import Keras.
from matplotlib import pyplot
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, Activation
from keras.layers import GlobalMaxPooling2D,GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras import initializers
from keras.optimizers import Adam
from keras.optimizers import rmsprop
from keras.layers.advanced_activations import LeakyReLU, PReLU
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping,ReduceLROnPlateau

from keras.datasets import cifar10
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from keras.applications.xception import Xception
from keras.applications.mobilenet import MobileNet
from keras.applications.vgg19 import VGG19
from keras.layers import Concatenate, Dense, LSTM, Input, concatenate
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input

#Data Aug for multi-input
from keras.preprocessing.image import ImageDataGenerator


In [ ]:
import pandas as pd 
import numpy as np 
import cv2 # Used to manipulated the images 
np.random.seed(1337) # The seed I used - pick your own or comment out for a random seed. A constant seed allows for better comparisons though

In [ ]:
input_path = "F:/Kaggle/Statoil/inputs/"
batch_size = 32

In [ ]:
train = pd.read_json(input_path+"train.json")
target_train=train['is_iceberg']
test = pd.read_json(input_path+"test.json")

In [ ]:
target_train=train['is_iceberg']
test['inc_angle']=pd.to_numeric(test['inc_angle'], errors='coerce')
train['inc_angle']=pd.to_numeric(train['inc_angle'], errors='coerce')#We have only 133 NAs.
train['inc_angle']=train['inc_angle'].fillna(method='pad')
X_angle=train['inc_angle']
test['inc_angle']=pd.to_numeric(test['inc_angle'], errors='coerce')
X_test_angle=test['inc_angle']

In [ ]:
def get_scaled_imgs(df):
    imgs = []
    
    for i, row in df.iterrows():
        #make 75x75 image
        band_1 = np.array(row['band_1']).reshape(sz, sz)
        band_2 = np.array(row['band_2']).reshape(sz, sz)
        band_3 = band_1 + band_2 # plus since log(x*y) = log(x) + log(y)
        
        # Rescale
        a = (band_1 - band_1.mean()) / (band_1.max() - band_1.min())
        b = (band_2 - band_2.mean()) / (band_2.max() - band_2.min())
        c = (band_3 - band_3.mean()) / (band_3.max() - band_3.min())

        imgs.append(np.dstack((a, b, c)))

    return np.array(imgs)

In [ ]:
sz = 75
X_train = get_scaled_imgs(train)
X_test = get_scaled_imgs(test)

In [ ]:
X_train.shape

In [ ]:
print("shape of train: {} and test: {}" .format(X_train.shape,X_test.shape))

In [ ]:
# Define the image transformations here
gen = ImageDataGenerator(horizontal_flip = True,
                         vertical_flip = True,
                         width_shift_range = 0.,
                         height_shift_range = 0.,
                         channel_shift_range=0,
                         zoom_range = 0.2,
                         rotation_range = 10)

# Here is the function that merges our two generators
# We use the exact same generator with the same random seed for both the y and angle arrays
def gen_flow_for_two_inputs(X1, X2, y):
    genX1 = gen.flow(X1,y,  batch_size=batch_size,seed=55)
    genX2 = gen.flow(X1,X2, batch_size=batch_size,seed=55)
    while True:
            X1i = genX1.next()
            X2i = genX2.next()
            #Assert arrays are equal - this was for peace of mind, but slows down training
            #np.testing.assert_array_equal(X1i[0],X2i[0])
            yield [X1i[0], X2i[1]], X1i[1]

def gen_flow_for_test(X1, X2,y=None):
    genX12 = gen.flow(X1,y, batch_size=batch_size,seed=55)
    genX22 = gen.flow(X1,X2, batch_size=batch_size,seed=55)
    while True:
            X21i = genX12.next()
            X22i = genX22.next()
            #Assert arrays are equal - this was for peace of mind, but slows down training
            #np.testing.assert_array_equal(X1i[0],X2i[0])
            yield [X21i[0], X22i[1]]
            
# Finally create generator
def get_callbacks(filepath, patience=2):
    es = EarlyStopping('val_loss', patience=10, mode="min")
    msave = ModelCheckpoint(filepath, save_best_only=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,patience=5, min_lr=0.01)
    return [es, msave,reduce_lr]

In [ ]:
# Define CNN Model Architecture (Kaggle can't access the weights file)
img_height = 224
img_width = 224
img_channels = 3
img_dim = (img_height, img_width, img_channels)

def inceptionv3(img_dim=img_dim):
    input_2 = Input(shape=[1], name="angle")
    angle_layer = Dense(1, )(input_2)
    input_tensor = Input(shape=img_dim)
    base_model = InceptionV3(include_top=False,
                   weights='imagenet',
                   input_shape=img_dim)
    bn = BatchNormalization()(input_tensor)
    x = base_model(bn)
    x = GlobalAveragePooling2D()(x)
    merge_one = concatenate([x, angle_layer])
    merge_one = Dense(512, activation='relu', name='fc2')(merge_one)
    merge_one = Dropout(0.3)(merge_one)
    merge_one = Dense(512, activation='relu', name='fc3')(merge_one)
    merge_one = Dropout(0.3)(merge_one)
    output = Dense(1, activation='sigmoid')(merge_one)
    
    model = Model(input=[input_tensor, input_2], output=output)
    
    return model

model = inceptionv3()
model.summary()

In [ ]:
# Train Model and predict
def train_model(model, batch_size, epochs, img_size, x, y, test, n_fold, kf,X_test_angle=X_test_angle):
        
    train_scores = []; valid_scores = []
    preds_test = np.zeros(len(test), dtype = np.float)

    i = 1

    for train_index, test_index in kf.split(x):
        x_train = x[train_index]; x_valid = x[test_index]
        y_train = y[train_index]; y_valid = y[test_index]

        #Angle
        X_angle_train = X_angle[train_index]
        X_angle_valid = X_angle[test_index]
        X_test_angle = X_test_angle

        def augment(src, choice):
            if choice == 0:
                # Rotate 90
                src = np.rot90(src, 1)
            if choice == 1:
                # flip vertically
                src = np.flipud(src)
            if choice == 2:
                # Rotate 180
                src = np.rot90(src, 2)
            if choice == 3:
                # flip horizontally
                src = np.fliplr(src)
            if choice == 4:
                # Rotate 90 counter-clockwise
                src = np.rot90(src, 3)
            if choice == 5:
                # Rotate 180 and flip horizontally
                src = np.rot90(src, 2)
                src = np.fliplr(src)
            return src

        def train_generator():
            while True:
                for start in range(0, len(x_train), batch_size):
                    x_batch = []
                    end = min(start + batch_size, len(x_train))
                    y_batch = y_train[start:end]
                    tr_angle = X_angle_train[start:end]
                    for img in x_train[start:end]:
                        new_img = cv2.resize(img, img_size)
                        new_img = augment(new_img, np.random.randint(6))
                        x_batch.append(new_img)
                    x_batch = np.array(x_batch, np.float32) / 255.
                    y_batch = np.array(y_batch, np.uint8)
                    yield [x_batch,tr_angle], y_batch

        def valid_generator():
            while True:
                for start in range(0, len(x_valid), batch_size):
                    x_batch = []
                    end = min(start + batch_size, len(x_valid))
                    y_batch = y_valid[start:end]
                    valid_angle = X_angle_valid[start:end]
                    for img in x_valid[start:end]:
                        new_img = cv2.resize(img, img_size)
                        x_batch.append(new_img)
                    x_batch = np.array(x_batch, np.float32) / 255.
                    y_batch = np.array(y_batch, np.uint8)
                    yield [x_batch ,valid_angle], y_batch

        def test_generator():
            while True:
                for start in range(0, len(test), n_fold):
                    x_batch = []
                    end = min(start + n_fold, len(test))
                    X_test_angle_test = X_test_angle[start:end]
                    for img in test[start:end]:
                        new_img = cv2.resize(img, img_size)
                        new_img = augment(new_img, np.random.randint(6))  ### TTA step
                        x_batch.append(new_img)
                    x_batch = np.array(x_batch, np.float32) / 255.
                    yield [x_batch,X_test_angle_test]
                    
        callbacks = [EarlyStopping('val_loss', patience=10, mode="min"),
                    ReduceLROnPlateau(monitor='val_loss', factor=0.2,patience=5, min_lr=0.01),
                    ModelCheckpoint(filepath='inception.fold_' + str(i) + '.hdf5', verbose=1,
                    save_best_only=True, save_weights_only=True, mode='auto')]

        train_steps = len(x_train) / batch_size
        valid_steps = len(x_valid) / batch_size
        test_steps = len(test) / n_fold
        
        model = model

        model.compile(optimizer=Adam(lr=1e-3), loss='binary_crossentropy', metrics = ['accuracy'])

        model.fit_generator(train_generator(), train_steps, epochs=epochs, verbose=1, 
                            callbacks=callbacks, validation_data=valid_generator(), 
                            validation_steps=valid_steps)

        model.load_weights(filepath='inception.fold_' + str(i) + '.hdf5')

        
        print('----------------------------------------')
        print('Running train evaluation on fold {}'.format(i))
        train_score = model.evaluate_generator(train_generator(), steps=train_steps)        
        print('Running validation evaluation on fold {}'.format(i))
        valid_score = model.evaluate_generator(valid_generator(), steps=valid_steps)
        print('----------------------------------------')   
        
        print('Train loss: {:0.5f}\n Train acc: {:0.5f} for fold {}'.format(train_score[0],
                                                                            train_score[1], i))
        print('Valid loss: {:0.5f}\n Valid acc: {:0.5f} for fold {}'.format(valid_score[0],
                                                                            valid_score[1], i))
        print('----------------------------------------')

        train_scores.append(train_score[1])
        valid_scores.append(valid_score[1])
        print('Avg Train Acc: {:0.5f}\nAvg Valid Acc: {:0.5f} after {} folds'.format
              (np.mean(train_scores), np.mean(valid_scores), i))
        print('----------------------------------------')
        
        print('Running test predictions with fold {}'.format(i))        
        preds_test_fold = model.predict_generator(generator=test_generator(),
                                              steps=test_steps, verbose=1)[:, -1]

        preds_test += preds_test_fold

        print('\n\n')

        i += 1

        if i <= n_fold:
            print('Now beginning training for fold {}\n\n'.format(i))
        else:
            print('Finished training!')

    preds_test /= n_fold

    return preds_test

In [ ]:
batch_size = 32
epochs = 50
n_fold = 5
img_size = (img_height, img_width)
kf = KFold(n_splits=n_fold, shuffle=True)

prediction_all_folds = train_model(model, batch_size, epochs, img_size, X_train, 
                                target_train, X_test, n_fold, kf,X_test_angle=X_test_angle)

In [ ]:
#Submission
import datetime as dt

submission = pd.DataFrame()
submission['id']=test['id']
submission['is_iceberg']=prediction_all_folds
submission.to_csv('F:/Kaggle/Statoil/submit/sub5cv_inceptionv3_tta_slight'+dt.datetime.today().strftime("%d%m%Y%H%M")+'.csv', index=False)